In [1]:
from omegaconf import OmegaConf
from pymongo import MongoClient
import pandas as pd

In [2]:
# to run the docker server: docker run --name testing-mongo -p 27017:27017 mongo:latest

def mongo_client():
    """Funcion to turn mongo client ON"""
    config = OmegaConf.load('/home/laertio/projects/dev/tv_scan/conf/mongo-config.yaml')
    server_ip = config.main.server_ip
    port = config.main.port
    # user = config.main.user
    # passwd = config.main.passwd
    client = MongoClient(f'mongodb://{server_ip}:{port}')
    #                     username=user,
    #                     password=passwd
    return client.TV_scan

In [ ]:
def core_processor_helper(norm, lang_detec, token, coll_name):
    """Helper to run the core text processing task defined in the processing_pipe function"""
    cursor = nlp_tank[coll_name].find()
    updates = []
    for doc in cursor:
        update = {}
        text = doc['text_raw']
        #text normalization
        if norm:
            text = normalize_text(text)
            update['text_norm_first'] = text
            #language detection
            if lang_detec:
                lang = detect_lang(text)
                update['text_lang_label'] = lang[0]
                update['text_lang_result'] = lang[1]

        #tokenization
        if token:
            if lang_detec: #only tokenize portuguese
                if "pt" in update['text_lang_label']:
                    tokens = text_tokenizer(text)
                    update['token_text_norm_first'] = tokens
                else:
                    label = update['text_lang_label']
                    logger.warning("Det. language: %s. Tokenization only for PT", label)
            else:
                ld_not_proc_before = field_checker(coll_name, 'text_lang_label')
                if ld_not_proc_before:
                    logger.error("Cannot Tokenize if no Language_detection.")
                    return print("Cannot Tokenize if no Language_detection.")
                if "pt" in doc['text_lang_label']:
                    tokens = text_tokenizer(text)
                    update['token_text_norm_first'] = tokens
                else:
                    label = update['text_lang_label']
                    logger.warning("Det. language: %s. Tokenization only for PT", label)

        updates.append(UpdateOne({'_id': doc['_id']}, {'$set': update}))

    cursor.close()
    return updates

In [5]:
df = pd.read_excel('./components/spotreg_update/data_in/input_sample.xlsx')

In [ ]:
df.info()

In [ ]:
df.columns

In [6]:
db = mongo_client()

In [10]:
insert = df.head().to_dict(orient='index') #records

In [11]:
insert = list(insert.values())
db['test_collection'].insert_many(insert)

In [12]:
cursor = db['test_collection'].find()

In [13]:
for item in cursor:
    print(item)

{'_id': ObjectId('63cfd28fd58793cd83de42a8'), 'MediaDesc': 'Televisão', 'SolDate': 20220101, 'SolDateArch': 20220224, 'SolDatePub': 20220101, 'SpotCode': 412561, 'SpotDesc': 'SABADO E DOMINGO 11/15H TEATRO POLITEAMA.RTP', 'SitCampId': 2, 'SitCampDesc': 'Nova Campanha', 'VehicleDesc': 'RTP1', 'BrandDesc': 'A PEQUENA SEREIA (MUSICAL)', 'SubBrandDesc': nan, 'SectorDesc': 'ARTIGOS E SERVICOS RECREATIVOS E CULTURAIS', 'CategoryDesc': 'ESPECTACULOS', 'ClassDesc': 'TEATROS', 'SubClassDesc': nan, 'AdvertiserDesc': 'BOCA DE CENA-PROD.ARTISTICAS LDA', 'MediaFile': 412561, 'MediaFileOldUrl': 'http://e-sol.mediamonitor.pt/SOLFILES/TELEVISAO/20220101/412561.wmv'}
{'_id': ObjectId('63cfd28fd58793cd83de42a9'), 'MediaDesc': 'Televisão', 'SolDate': 20220101, 'SolDateArch': 20220224, 'SolDatePub': 20220101, 'SpotCode': 412475, 'SpotDesc': 'SABADO E DOMINGO 11/15H TEATRO POLITEAMA', 'SitCampId': 2, 'SitCampDesc': 'Nova Campanha', 'VehicleDesc': 'Panda', 'BrandDesc': 'A PEQUENA SEREIA (MUSICAL)', 'SubBran

In [15]:
result = db['test_collection'].find_one(sort=[("SolDate", -1)])

In [21]:
result['SolDate']

20220101

In [24]:
filter = df['SolDate'].gt(20220101)
df.loc[filter]

,MediaDesc,SolDate,SolDateArch,SolDatePub,SpotCode,SpotDesc,SitCampId,SitCampDesc,VehicleDesc,BrandDesc,SubBrandDesc,SectorDesc,CategoryDesc,ClassDesc,SubClassDesc,AdvertiserDesc,MediaFile,MediaFileOldUrl
71,Televisão,20220102,20220224,20220102,417251,"LUPILU:FRALDAS 4,59E/TOALHITAS 0,79E",4,Novo Filme,TVI,LIDL,NaN,COMERCIO,RETALHO,SUPER E HIPER MERCADOS,GENERALISTAS,LOJAS LIDL COMPANHIA,417251,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
72,Televisão,20220102,20220224,20220102,417252,ENTREGA GRATIS:MAQUINA LAVAR ROUPA SAMSUNG 399E,4,Novo Filme,CMTV,WORTEN,NaN,COMERCIO,RETALHO,SUPER E HIPER MERCADOS,ESPECIALIDADES,WORTEN-SONAE SR,417252,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
73,Televisão,20220102,20220224,20220102,417253,ENTREGA GRATIS:XIAOMI 11 LITE 5G 399E,4,Novo Filme,TVI,WORTEN,NaN,COMERCIO,RETALHO,SUPER E HIPER MERCADOS,ESPECIALIDADES,WORTEN-SONAE SR,417253,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
74,Televisão,20220102,20220224,20220102,417254,"PRECOS BAIXOS:DOURADA FRESCA 5,49E/60%SKIP LIQ...",4,Novo Filme,RTP1,CONTINENTE,NaN,COMERCIO,RETALHO,SUPER E HIPER MERCADOS,GENERALISTAS,MODELO CONTINENTE HIPERMERCADOS SA,417254,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
75,Televisão,20220102,20220224,20220102,417255,DEMASIADA ROUPA A OCUPAR ESPACO?FAZ JA DOWNLOA...,4,Novo Filme,SIC,VINTED,NaN,COMERCIO,VENDAS POR INTERNET,NaN,NaN,VINTED UAB,417255,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14668,Televisão,20221226,20221229,20221226,434496,ESTE PR(BRITANS TALENT)E PAT.POR SECONDLOVE.PT...,4,Novo Filme,SIC Mulher,SECONDLOVE.PT,NaN,SERVICOS PESSOAIS,OUTROS SERVICOS PESSOAIS,NaN,NaN,UTELL DIGITAL SL,434496,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
14669,Televisão,20221228,20221229,20221228,428104,UMA EXPLOSAO DE SABOR...FAZ PLAY(SOFA/3EMB),2,Nova Campanha,SIC,DORITOS,NaN,INDUSTRIA ALIMENTACAO,APERITIVOS,NaN,NaN,MATUDIS,428104,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
14670,Televisão,20221228,20221229,20221228,429751,PAT.VOLANTE-E SKODA ENYAQ IV(SPORTLINE),2,Nova Campanha,SIC Noticias,SKODA,ENYAQ SPORTLINE IV,INDUSTRIA AUTOMOVEL,VEICULOS A MOTOR,AUTOMOVEIS,AUTOMOVEIS DE PASSAGEIROS,SIVA-SOC.IMPORTADORA VEICULOS AUTOMOVEIS,429751,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
14671,Televisão,20221228,20221229,20221228,430690,PAT.TODA A VERDADE-ARCADA IMOBILIARIA JUNTE-SE...,2,Nova Campanha,SIC Noticias,ARCADA IMOBILIARIA,NaN,OPERACOES SOBRE IMOVEIS,AGENCIAS PREDIAIS E IMOBILIARIAS,NaN,NaN,ARCADA GLICINIAS,430690,http://e-sol.mediamonitor.pt/SOLFILES/TELEVISA...
